<a href="https://colab.research.google.com/github/TomographicImaging/gVXR-Tutorials/blob/main/notebooks/multi_material-lungman_phantom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
#
#  Copyright 2024 United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Franck Vidal (UKRI-STFC)

![gVXR](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/Logo-transparent-small.png?raw=1)

# Multi-material example: Lungman phantom


<div class="alert alert-block alert-warning">
    <b>Note:</b> Make sure the Python packages are already installed. See <a href="../README.md">README.md</a> in the root directory of the repository. If you are running this notebook from Google Colab, please run the cell below to install the required packages.
</div>

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install gvxr

# Aims of this session

1. Create our first X-ray simulation, step-by-step;
2. Save our X-ray image in a file format that preserves the original dynamic range;
3. Visualise the results with 3 different look-up tables;
4. Visualise the 3D environment.

![](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/02-visualisation.png?raw=1)

# Main steps

There are 6 main steps to simulate an X-ray image:

1. Create a renderer (OpenGL context)
2. Set the X-ray source
3. Set the Spectrum
4. Set the Detector
5. Set the Sample
6. Compute the corresponding X-ray image.

![](https://github.com/TomographicImaging/gVXR-Tutorials/blob/main/img/dragon2.jpg?raw=1)

# Cheat Sheet

## Simulation parameters

| Parameters | Values | Units | Function call |
|------------|--------|-------|---------------|
| Source position | [-40, 0, 0] | cm | `gvxr.setSourcePosition(-40.0, 0.0, 0.0, "cm")` |
| Source type (beam shape) | Point source (cone beam) | | `gvxr.usePointSource()` |
| Beam spectrum | Monochromatic: 1,000 photons of 80 | keV | `gvxr.setMonoChromatic(80.0, "keV", 1000)` or |
| | | MeV | `gvxr.setMonoChromatic(0.08, "MeV", 1000)` |
| Detector position | [10, 0, 0] | cm | `gvxr.setDetectorPosition(10.0, 0.0, 0.0, "cm")` |
| Detector orientation | [0, 0, -1] |  | `gvxr.setDetectorUpVector(0, 0, -1)` |
| Detector resolution | 640 &times; 320 | pixels | `gvxr.setDetectorNumberOfPixels(640, 320)` |
| Pixel spacing | 0.5 &times; 0.5 | mm | `gvxr.setDetectorPixelSize(0.5, 0.5, "mm")` |
| Sample | STL file of the Welsh dragon | mm | `gvxr.loadMeshFile(string_ID, fname, "mm")` |
| Material composition | Ti90Al6V4 alloy |  | `gvxr.setMixture("Dragon", "Ti90Al6V4")` or |
| | | | `gvxr.setMixture("Dragon", [22, 13, 23], [0.9, 0.06, 0.04])` |
| Material density | 4.43 | g/cm3 | `gvxr.setDensity("Dragon", 4.43, "g/cm3")` or |
| | | g.cm-3 | `gvxr.setDensity("Dragon", 4.43, "g.cm-3")` |

## Other gVXR's function used

| Functions | Descriptions |
|-----------|--------------|
| `gvxr.createOpenGLContext` | Create a simulation environment automatically. You must call either `gvxr.createOpenGLContext()` or `gvxr.createNewContext()` before any other functions from gVXR. |
| `gvxr.moveToCentre` | Move a polygon mesh to the centre. |
| `gvxr.setElement` | Set the chemical element (e.g. iron) corresponding to the material properties of a polygon mesh. |
| `gvxr.setCompound` | Set the compound (e.g. water) corresponding to the material properties of a polygon mesh. |
| `gvxr.computeXRayImage` | Compute the X-ray projection corresponding to the environment that has previously been set. |
| `gvxr.displayScene` | Update the visualisation window. It works with both the interactive window and offscreen rendering. |
| `gvxr.saveLastXRayImage` | Save the last computed X-ray image in a file (e.g. TIFF, MHA, MHD/RAW) and store the data using single-precision floating-point numbers. |
| `gvxr.saveLastLBuffer` | Save the last computed path length in a file (e.g. TIFF, MHA, MHD/RAW) and store the data using single-precision floating-point numbers. The red channel is the path length; the green and blue channels can be used to detect errors (0.0 values mean no error). |
| `gvxr.setColour` | Change the colour of an object in the 3D visualisation. |
| `gvxr.setWindowBackGroundColour` | Change the background colour of the 3D visualisation. |
| `gvxr.takeScreenshot` | Take the screenshot of the current state of the 3D visualisation. |
| `gvxr.renderLoop` | Activate the interactive mode of the 3D visualisation. |
| `gvxr.terminate` | Close and destroy all the windows and simulation contexts that have been created. No further gVXR's function should be called after `gvxr.terminate`. |

## Import packages

- `os` to create the output directory if needed
- `matplotlib` to show 2D images
- `tifffile` to write TIFF files
- `gvxr` to simulate X-ray images

In [ ]:
import os # Create the output directory if necessary
import numpy as np # Who does not use Numpy?

import matplotlib # To plot images
import matplotlib.pyplot as plt # Plotting
from matplotlib.colors import LogNorm # Look up table
from matplotlib.colors import PowerNorm # Look up table

font = {'family' : 'serif',
         'size'   : 15
       }
matplotlib.rc('font', **font)

# Uncomment the line below to use LaTeX fonts
# matplotlib.rc('text', usetex=True)

from tifffile import imwrite # Write TIFF files

import urllib.request
import progressbar

import zipfile

from gvxrPython3 import gvxr
from gvxrPython3.utils import visualise

In [ ]:
output_path = "output_data/multi_material-lungman_phantom"
if not os.path.exists(output_path):
    os.makedirs(output_path);

lungman_path = os.path.join("output_data", "lungman");
mesh_path = os.path.join(lungman_path, "MESHES");
if not os.path.exists(mesh_path):
    os.makedirs(mesh_path);    

Download the data from [Zenodo](https://zenodo.org/records/10782644).

In [ ]:
zip_file_url = "https://zenodo.org/records/10782644/files/lungman_data.zip?download=1";

zip_fname = os.path.join(lungman_path, "lungman_data.zip");

if not os.path.exists(zip_fname):
    pbar = None;
    def show_progress(block_num, block_size, total_size):
        global pbar
        if pbar is None:
            pbar = progressbar.ProgressBar(maxval=total_size)
            pbar.start()
    
        downloaded = block_num * block_size
        if downloaded < total_size:
            pbar.update(downloaded)
        else:
            pbar.finish()
            pbar = None
            
    print("Download the file (%s) from %s\n" % (zip_fname, zip_file_url))
    urllib.request.urlretrieve(zip_file_url, zip_fname, show_progress)

Extract the STL files from the ZIP file.

In [ ]:
stl_fname_set = [];

with zipfile.ZipFile(zip_fname) as z:
    for fname in z.namelist():
        if ".stl" in fname:
            stl_fname = os.path.join(lungman_path, fname);
            stl_fname_set.append(stl_fname);
            
            if not os.path.exists(stl_fname):
                print("Extract %s" % stl_fname);
                with open(stl_fname, 'wb') as f:
                    f.write(z.read(fname));

## 1. Create an OpenGL context

The first step is to create the simulation environment, known here as "OpenGL context".
`createOpenGLContext` will try to find the most suitable environment possible regardless of the operating system. This is an alternative function to `gvxr.createNewContext` used in [test_installation.ipynb](test_installation.ipynb).

In [ ]:
# Create an OpenGL context
print("Create an OpenGL context");
gvxr.createOpenGLContext();

## 2. Set the X-ray source

We must set it's possition and beam shape.

In [ ]:
# Create a source
print("Set up the beam");
gvxr.setSourcePosition(0.0, -80.0, 0.0, "cm");
gvxr.usePointSource();
#  For a parallel source, use gvxr.useParallelBeam();

## 3. Set the Spectrum

We define here the number of photons and their kinetic energy.

In [ ]:
# Set its spectrum, here a monochromatic beam
# 1000 photons of 80 keV (i.e. 0.08 MeV) per ray
gvxr.setMonoChromatic(0.08, "MeV", 1000);
# The following is equivalent: gvxr.setMonoChromatic(80, "keV", 1000);

## 4. Set the Detector

A detector is defined by its position, orientation, pixel resolution and the space between the centre of two consecutive pixels along its two axes.

In [ ]:
# Set up the detector
print("Set up the detector");
gvxr.setDetectorPosition(0.0, 30.0, 0.0, "cm");
gvxr.setDetectorUpVector(0, 0, 1);
gvxr.setDetectorNumberOfPixels(2000, 2000);
gvxr.setDetectorPixelSize(0.25, 0.25, "mm");

## 5. Set the Sample

A sample is define by its geometry (surface) and material composition. Note that you can transform (translate, scale and rotate) a sample.

In [ ]:
gvxr.removePolygonMeshesFromXRayRenderer();

geometry_set = {
    "bronchioles": {"HU": -419.57144, "Colour" : [0, 240, 240, 0.08]},
    "bronchus": {"HU": -40.36795, "Colour" : [0, 62, 186, 0.4]},
    "trachea": {"HU": -914.32916, "Colour" : [170, 85, 255, 0.4]},
    "diaphram": {"HU": -12.778751, "Colour" : [255, 85, 127, 1]},
    "skin": {"HU": -12.121676, "Colour" : [125, 125, 125, 0.17]},
    "heart": {"HU": 28.384626, "Colour" : [255, 0, 0, 1]},
    "sheets_low": {"HU": -158.2706, "Colour" : [193, 193, 193, 1]},
    "sheets_med": {"HU": 203.39578, "Colour" : [193, 193, 193, 1]},
    "sheets_high": {"HU": 324.9135, "Colour" : [193, 193, 193, 1]},
    "tumours_630HU": {"HU": -658.61346, "Colour" : [138, 0, 0, 1]},
    "tumours_100HU": {"HU": 83.32481, "Colour" : [255, 85, 0, 1]},
    "spine-hard-650": {"HU": 857.8602, "Colour" : [255, 255, 127, 1]},
    "spine-soft-650": {"HU": 375.58865, "Colour" : [255, 255, 127, 1]},
    "scaps-hard-550": {"HU": 709.09717, "Colour" : [255, 255, 127, 1]},
    "scaps-soft-550": {"HU": 372.82138, "Colour" : [255, 255, 127, 1]},
    "sternum-hard-550": {"HU": 789.6037, "Colour" : [255, 255, 127, 1]},
    "sternum-soft-550": {"HU": 378.79736, "Colour" : [255, 255, 127, 1]},
    "clavicle-hard-700": {"HU": 778.28, "Colour" : [255, 255, 127, 1]},
    "clavicle-soft-700": {"HU": 261.89047, "Colour" : [255, 255, 127, 1]},
}

for label in geometry_set:
    if "sheet" not in label:
        gvxr.loadMeshFile(label, os.path.join(mesh_path, label + ".stl"), "mm");
        gvxr.setHounsfieldUnit(label, round(geometry_set[label]["HU"]));
        gvxr.setColour(label,
            geometry_set[label]["Colour"][0] / 255.0,
            geometry_set[label]["Colour"][1] / 255.0,
            geometry_set[label]["Colour"][0] / 255.0,
            geometry_set[label]["Colour"][3]);

## 6. Compute the corresponding X-ray image.

It is possible to compute, retrieve and save an X-ray image as well as the path length of X-ray through an object.

In [ ]:
# Compute an X-ray image
# We convert the array in a Numpy structure and store the data using single-precision floating-point numbers.
print("Compute an X-ray image");
x_ray_image = gvxr.computeXRayImage();

# Update the visualisation window
gvxr.displayScene();

In [ ]:
# Save the X-ray image in a TIFF file and store the data using single-precision floating-point numbers.
gvxr.saveLastXRayImage(os.path.join(output_path, 'raw_x-ray_image-01.tif'));

# The line below will also works
imwrite(os.path.join(output_path, 'raw_x-ray_image-02.tif'), x_ray_image);

# Save the L-buffer
gvxr.saveLastLBuffer(os.path.join(output_path, 'path_length-dragon.tif'));

A linear look up table is not always the best way to visualise X-ray images. A log-scale may be favoured (but be mindful $\log(0)$ does not exist. Another popular method is the Power-law colour scale as we can control the Power law exponent ($\gamma$). The corresponding normalisation formula is as follows:
$$\left(\frac{Img - \min(Img)}{\max(Img) - \min(img)}\right)^\gamma$$

In [ ]:
# Display the X-ray image and compare three different lookup tables
plt.figure(figsize=(15, 7.5));

plt.suptitle("Image simulated with gVirtualXray visualised", y=0.95);

plt.subplot(131);
plt.imshow(x_ray_image, cmap="gray");
plt.colorbar(orientation='horizontal');
plt.title("using a linear colour scale");

plt.subplot(132);
plt.imshow(x_ray_image, norm=LogNorm(), cmap="gray");
plt.colorbar(orientation='horizontal');
plt.title("using a logarithmic colour scale");

plt.subplot(133);
plt.imshow(x_ray_image, norm=PowerNorm(gamma=1./2.), cmap="gray");
plt.colorbar(orientation='horizontal');
plt.title("using a Power-law colour scale ($\gamma=0.5$)");

plt.tight_layout();
plt.savefig(os.path.join(output_path, 'projection.pdf'), dpi=600);
plt.show();

It may be useful to visualise the 3D environment to ascertain everything is as expected. It can be done in a window or offscreen with a screenshot.

In [ ]:
# This image can be used in a research paper to illustrate the simulation environment, in which case you may want to change the background colour to white with:
gvxr.setWindowBackGroundColour(1.0, 1.0, 1.0);

# Update the visualisation window
gvxr.displayScene();

In [ ]:
# Take the screenshot and save it in a file
screenshot = gvxr.takeScreenshot();
plt.imsave(os.path.join(output_path, "screenshot-01.png"), np.array(screenshot));

# or display it using Matplotlib
plt.figure(figsize=(10, 10));
plt.imshow(screenshot);
plt.title("Screenshot of the X-ray simulation environment");
plt.axis('off');
plt.show();

In [ ]:
print(gvxr.getZoom());
print(gvxr.getSceneRotationMatrix());

In [ ]:
gvxr.setZoom(1397.4293212890625)
gvxr.setSceneRotationMatrix([
    -0.8833954334259033, 0.017649494111537933, 0.46829527616500854, 0.0,
    -0.46784770488739014, 0.024374328553676605, -0.883472204208374, 0.0, 
    -0.02700728178024292, -0.9995452165603638, -0.01327541097998619, 0.0,
    0.0, 0.0, 0.0, 1.0]);

# Update the visualisation window
gvxr.displayScene();

# Take the screenshot and save it in a file
screenshot = gvxr.takeScreenshot();
plt.imsave(os.path.join(output_path, "screenshot-02.png"), np.array(screenshot));

# or display it using Matplotlib
plt.figure(figsize=(10, 10));
plt.imshow(screenshot);
plt.title("Screenshot of the X-ray simulation environment");
plt.axis('off');
plt.show();

If windowing is possible, it is possible to run the 3D visualisation in an interactive window.

In [ ]:
# Interactive visualisation
# The user can rotate the 3D scene and zoom-in and -out in the visualisation window.

# - Keys are:
#     - Q/Escape: to quit the event loop (does not close the window)
#     - B: display/hide the X-ray beam
#     - W: display the polygon meshes in solid or wireframe
#     - N: display the X-ray image in negative or positive
#     - H: display/hide the X-ray detector
# - Mouse interactions:
#     - Zoom in/out: mouse wheel
#     - Rotation: Right mouse button down + move cursor```
# gvxr.renderLoop();

print(gvxr.getZoom());
print(gvxr.getSceneRotationMatrix());

In [ ]:
if not IN_COLAB:
    plot = visualise(use_log=True, use_negative=True);
    plot.display();

# Cleaning up

Once we have finished it is good practice to clean up the OpenGL contexts and windows with the following command.

In [ ]:
# gvxr.terminate();